# End-to-End Reproducibility Demo
This notebook provides a minimal, reproducible example that illustrates the full workflow of $LYRA$.

In [1]:
# !python3 -m venv venv-jupyterhub
# !bash -c "source venv-jupyterhub/bin/activate && pip install ipykernel && python -m ipykernel install --user --name='python3-user-venv' --display-name 'Python 3 (venv-user)'"
# %pip install sentence-transformers
# %pip install scipy
# %pip install gymnasium
# %pip install pettingzoo
# %pip install catboost
# %pip install ontolearn
# %pip install owlapy
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
# %pip install tensorboard

In [2]:
from sentence_transformers import SentenceTransformer, models


class ExpressionEncoder:
    def __init__(self, device, model_name='all-mpnet-base-v2'):
        self.model = SentenceTransformer(model_name,device = device)
        embedding_dim = self.model.get_sentence_embedding_dimension()
        projection = models.Dense(in_features=embedding_dim, out_features=64)
        self.model.add_module('projection', projection)

    def encode(self, sentences):
        return self.model.encode(sentences, convert_to_tensor=True)

# Mass functions

In [3]:
"""
Most of the code is taken from this repo: https://github.com/reineking/pyds
We did a little maintenance and added extra functionalities
"""

from __future__ import print_function
from itertools import chain, combinations
from functools import partial, reduce
from operator import mul
from math import log, fsum, sqrt
from random import random, shuffle, uniform
import sys

try:
    import numpy

    try:
        from scipy.stats import chi2
        from scipy.optimize import fmin_cobyla
    except:
        print('SciPy not found: some features will not work.', file=sys.stderr)
except:
    print('NumPy not found: some features will not work.', file=sys.stderr)


class MassFunction(dict):
    """
    A Dempster-Shafer mass function (basic probability assignment) based on a dictionary.

    Both normalized and unnormalized mass functions are supported.
    The underlying frame of discernment is assumed to be discrete.

    Hypotheses and their associated mass values can be added/changed/removed using the standard dictionary methods.
    Each hypothesis can be an arbitrary sequence which is automatically converted to a 'frozenset', meaning its elements must be hashable.
    """

    def __init__(self, source=None):
        """
        Creates a new mass function.

        If 'source' is not None, it is used to initialize the mass function.
        It can either be a dictionary mapping hypotheses to non-negative mass values
        or an iterable containing tuples consisting of a hypothesis and a corresponding mass value.
        """
        if source != None:
            if isinstance(source, dict):
                source = source.items()
            for (h, v) in source:
                self[h] += v

    @staticmethod
    def _convert(hypothesis):
        """Convert hypothesis to a 'frozenset' in order to make it hashable."""
        if isinstance(hypothesis, frozenset):
            return hypothesis
        else:
            return frozenset(hypothesis)

    @staticmethod
    def gbt(likelihoods, normalization=True, sample_count=None):
        """
        Constructs a mass function using the generalized Bayesian theorem.
        For more information, see Smets. 1993. Belief functions:
        The disjunctive rule of combination and the generalized Bayesian theorem. International Journal of Approximate Reasoning.

        'likelihoods' specifies the conditional plausibilities for a set of singleton hypotheses.
        It can either be a dictionary mapping singleton hypotheses to plausibilities or an iterable
        containing tuples consisting of a singleton hypothesis and a corresponding plausibility value.

        'normalization' determines whether the resulting mass function is normalized, i.e., whether m({}) == 0.

        If 'sample_count' is not None, the true mass function is approximated using the specified number of samples.
        """
        m = MassFunction()
        if isinstance(likelihoods, dict):
            likelihoods = list(likelihoods.items())
        # filter trivial likelihoods 0 and 1
        ones = [h for (h, l) in likelihoods if l >= 1.0]
        likelihoods = [(h, l) for (h, l) in likelihoods if 0.0 < l < 1.0]
        if sample_count == None:  # deterministic
            def traverse(m, likelihoods, ones, index, hyp, mass):
                if index == len(likelihoods):
                    m[hyp + ones] = mass
                else:
                    traverse(m, likelihoods, ones, index + 1, hyp + [likelihoods[index][0]],
                             mass * likelihoods[index][1])
                    traverse(m, likelihoods, ones, index + 1, hyp, mass * (1.0 - likelihoods[index][1]))

            traverse(m, likelihoods, ones, 0, [], 1.0)
            if normalization:
                m.normalize()
        else:  # Monte-Carlo
            if normalization:
                empty_mass = reduce(mul, [1.0 - l[1] for l in likelihoods], 1.0)
            for _ in range(sample_count):
                rv = [random() for _ in range(len(likelihoods))]
                subtree_mass = 1.0
                hyp = set(ones)
                for k in range(len(likelihoods)):
                    l = likelihoods[k][1]
                    p_t = l * subtree_mass
                    p_f = (1.0 - l) * subtree_mass
                    if normalization and not hyp:  # avoid empty hypotheses in the normalized case
                        p_f -= empty_mass
                    if p_t > rv[k] * (p_t + p_f):
                        hyp.add(likelihoods[k][0])
                    else:
                        subtree_mass *= 1 - l  # only relevant for the normalized empty case
                m[hyp] += 1.0 / sample_count
        return m

    @staticmethod
    def from_bel(bel):
        """
        Creates a mass function from a corresponding belief function.

        'bel' is a dictionary mapping hypotheses to belief values (like the dictionary returned by 'bel(None)').
        """
        m = MassFunction()
        for h1 in bel.keys():
            v = fsum([bel[h2] * (-1) ** (len(h1 - h2)) for h2 in powerset(h1)])
            if v > 0:
                m[h1] = v
        mass_sum = fsum(m.values())
        if mass_sum < 1.0:
            m[frozenset()] = 1.0 - mass_sum
        return m

    @staticmethod
    def from_pl(pl):
        """
        Creates a mass function from a corresponding plausibility function.

        'pl' is a dictionary mapping hypotheses to plausibility values (like the dictionary returned by 'pl(None)').
        """
        frame = max(pl.keys(), key=len)
        bel_theta = pl[frame]
        bel = {frozenset(frame - h): bel_theta - v for (h, v) in
               pl.items()}  # follows from bel(-A) = bel(frame) - pl(A)
        return MassFunction.from_bel(bel)

    @staticmethod
    def from_q(q):
        """
        Creates a mass function from a corresponding commonality function.

        'q' is a dictionary mapping hypotheses to commonality values (like the dictionary returned by 'q(None)').
        """
        m = MassFunction()
        frame = max(q.keys(), key=len)
        for h1 in q.keys():
            v = fsum([q[h1 | h2] * (-1) ** (len(h2 - h1)) for h2 in powerset(frame - h1)])
            if v > 0:
                m[h1] = v
        mass_sum = fsum(m.values())
        if mass_sum < 1.0:
            m[frozenset()] = 1.0 - mass_sum
        return m

    def __missing__(self, key):
        """Return 0 mass for hypotheses that are not contained."""
        return 0.0

    def __copy__(self):
        c = MassFunction()
        for k, v in self.items():
            c[k] = v
        return c

    def copy(self):
        """Creates a shallow copy of the mass function."""
        return self.__copy__()

    def __contains__(self, hypothesis):
        return dict.__contains__(self, MassFunction._convert(hypothesis))

    def __getitem__(self, hypothesis):
        return dict.__getitem__(self, MassFunction._convert(hypothesis))

    def __setitem__(self, hypothesis, value):
        """
        Adds or updates the mass value of a hypothesis.

        'hypothesis' is automatically converted to a 'frozenset' meaning its elements must be hashable.
        In case of a negative mass value, a ValueError is raised.
        """
        if value < 0.0:
            raise ValueError("mass value is negative: %f" % value)
        dict.__setitem__(self, MassFunction._convert(hypothesis), value)

    def __delitem__(self, hypothesis):
        return dict.__delitem__(self, MassFunction._convert(hypothesis))

    def frame(self):
        """
        Returns the frame of discernment of the mass function as a 'frozenset'.

        The frame of discernment is the union of all contained hypotheses.
        In case the mass function does not contain any hypotheses, an empty set is returned.
        """
        if not self:
            return frozenset()
        else:
            return frozenset.union(*self.keys())

    def singletons(self):
        """
        Returns the set of all singleton hypotheses.

        Like 'frame()', except that each singleton is wrapped in a frozenset
        and can thus be directly passed to methods like 'bel()'.
        """
        return {frozenset((s,)) for s in self.frame()}

    def focal(self):
        """
        Returns the set of all focal hypotheses.

        A focal hypothesis has a mass value greater than 0.
        """
        return {h for (h, v) in self.items() if v > 0}

    def core(self, *mass_functions):
        """
        Returns the core of one or more mass functions as a 'frozenset'.

        The core of a single mass function is the union of all its focal hypotheses.
        In case a mass function does not contain any focal hypotheses, its core is an empty set.
        If multiple mass functions are given, their combined core (intersection of all single cores) is returned.
        """
        if mass_functions:
            return frozenset.intersection(self.core(), *[m.core() for m in mass_functions])
        else:
            focal = self.focal()
            if not focal:
                return frozenset()
            else:
                return frozenset.union(*focal)

    def all(self):
        """Returns an iterator over all subsets of the frame of discernment, including the empty set."""
        return powerset(self.frame())

    def bel(self, hypothesis=None):
        """
        Computes either the belief of 'hypothesis' or the entire belief function (hypothesis=None).

        If 'hypothesis' is None (default), a dictionary mapping hypotheses to their respective belief values is returned.
        Otherwise, the belief of 'hypothesis' is returned.
        In this case, 'hypothesis' is automatically converted to a 'frozenset' meaning its elements must be hashable.
        """
        if hypothesis is None:
            return {h: self.bel(h) for h in powerset(self.core())}
        else:
            hypothesis = MassFunction._convert(hypothesis)
            if not hypothesis:
                return 0.0
            else:
                return fsum([v for (h, v) in self.items() if h and hypothesis.issuperset(h)])

    def pl(self, hypothesis=None):
        """
        Computes either the plausibility of 'hypothesis' or the entire plausibility function (hypothesis=None).

        If 'hypothesis' is None (default), a dictionary mapping hypotheses to their respective plausibility values is returned.
        Otherwise, the plausibility of 'hypothesis' is returned.
        In this case, 'hypothesis' is automatically converted to a 'frozenset' meaning its elements must be hashable.
        """
        if hypothesis is None:
            return {h: self.pl(h) for h in powerset(self.core())}
        else:
            hypothesis = MassFunction._convert(hypothesis)
            if not hypothesis:
                return 0.0
            else:
                return fsum([v for (h, v) in self.items() if hypothesis & h])

    def q(self, hypothesis=None):
        """
        Computes either the commonality of 'hypothesis' or the entire commonality function (hypothesis=None).

        If 'hypothesis' is None (default), a dictionary mapping hypotheses to their respective commonality values is returned.
        Otherwise, the commonality of 'hypothesis' is returned.
        In this case, 'hypothesis' is automatically converted to a 'frozenset' meaning its elements must be hashable.
        """
        if hypothesis is None:
            return {h: self.q(h) for h in powerset(self.core())}
        else:
            if not hypothesis:
                return 1.0
            else:
                return fsum([v for (h, v) in self.items() if h.issuperset(hypothesis)])

    def __and__(self, mass_function):
        """Shorthand for 'combine_conjunctive(mass_function)'."""
        return self.combine_conjunctive(mass_function)

    def __or__(self, mass_function):
        """Shorthand for 'combine_disjunctive(mass_function)'."""
        return self.combine_disjunctive(mass_function)

    def __str__(self):
        hyp = sorted([(v, h) for (h, v) in self.items()], reverse=True)
        return "{" + "; ".join([str(set(h)) + ":" + str(v) for (v, h) in hyp]) + "}"

    def __mul__(self, scalar):
        if not isinstance(scalar, float):
            raise TypeError('Can only multiply by a float value.')
        m = MassFunction()
        for (h, v) in self.items():
            m[h] = v * scalar
        return m

    def __rmul__(self, scalar):
        return self.__mul__(scalar)

    def __add__(self, m):
        if not isinstance(m, MassFunction):
            raise TypeError('Can only add two mass functions.')
        result = self.copy()
        for (h, v) in m.items():
            result[h] += v
        return result

    def weight_function(self):
        """
        Computes the weight function corresponding to this mass function.
        """
        weights = dict()
        q = self.q()
        theta = self.frame()
        for h in powerset(theta):
            if len(h) < len(theta):  # weight is undefined for theta
                sets = [h | c for c in powerset(theta - h)]
                q_even = reduce(mul, [q[h2] for h2 in sets if len(h2) % 2 == 0], 1.0)
                q_odd = reduce(mul, [q[h2] for h2 in sets if len(h2) % 2 == 1], 1.0)
                if len(h) % 2 == 0:
                    weights[h] = q_odd / q_even
                else:
                    weights[h] = q_even / q_odd
        return weights

    def combine_conjunctive(self, mass_function, normalization=True, sample_count=None, importance_sampling=False):
        """
        Conjunctively combines the mass function with another mass function and returns the combination as a new mass function.

        The other mass function is assumed to be defined over the same frame of discernment.
        If 'mass_function' is not of type MassFunction, it is assumed to be an iterable containing multiple mass functions that are iteratively combined.

        If the mass functions are flatly contracting or if one of the mass functions is empty, an empty mass function is returned.

        'normalization' determines whether the resulting mass function is normalized (default is True).

        If 'sample_count' is not None, the true combination is approximated using the specified number of samples.
        In this case, 'importance_sampling' determines the method of approximation (only if normalization=True, otherwise 'importance_sampling' is ignored).
        The default method (importance_sampling=False) independently generates samples from both mass functions and computes their intersections.
        If importance_sampling=True, importance sampling is used to avoid empty intersections, which leads to a lower approximation error but is also slower.
        This method should be used if there is significant evidential conflict between the mass functions.
        """
        return self._combine(mass_function, rule=lambda s1, s2: s1 & s2, normalization=normalization,
                             sample_count=sample_count, importance_sampling=importance_sampling)

    def combine_disjunctive(self, mass_function, sample_count=None):
        """
        Disjunctively combines the mass function with another mass function and returns the combination as a new mass function.

        The other mass function is assumed to be defined over the same frame of discernment.
        If 'mass_function' is not of type MassFunction, it is assumed to be an iterable containing multiple mass functions that are iteratively combined.

        If 'sample_count' is not None, the true combination is approximated using the specified number of samples.
        """
        return self._combine(mass_function, rule=lambda s1, s2: s1 | s2, normalization=False, sample_count=sample_count,
                             importance_sampling=False)

    def combine_cautious(self, mass_function):
        """
        Combines the mass function with another mass function using the cautious rule and returns the combination as a new mass function.

        For more details, see:
        T. Denoeux (2008), "Conjunctive and disjunctive combination of belief functions induced by nondistinct bodies of evidence",
        Artificial Intelligence 172, 234-264.
        """
        w1 = self.weight_function()
        w2 = mass_function.weight_function()
        w_min = {h: min(w1[h], w2[h]) for h in w1}
        theta = self.frame()
        m = MassFunction({theta: 1.0})
        for h, w in w_min.items():
            m_simple = MassFunction({theta: w, h: 1.0 - w})
            m = m.combine_conjunctive(m_simple, normalization=False)
        return m

    def _combine(self, mass_function, rule, normalization, sample_count, importance_sampling):
        """Helper method for combining two or more mass functions."""
        combined = self
        if isinstance(mass_function, MassFunction):
            mass_function = [mass_function]  # wrap single mass function
        for m in mass_function:
            if not isinstance(m, MassFunction):
                raise TypeError(
                    "expected type MassFunction but got %s; make sure to use keyword arguments for anything other than mass functions" % type(
                        m))
            if sample_count == None:
                combined = combined._combine_deterministic(m, rule)
            else:
                if importance_sampling and normalization:
                    combined = combined._combine_importance_sampling(m, sample_count)
                else:
                    combined = combined._combine_direct_sampling(m, rule, sample_count)
        if normalization:
            return combined.normalize()
        else:
            return combined

    def _combine_deterministic(self, mass_function, rule):
        """Helper method for deterministically combining two mass functions."""
        combined = MassFunction()
        for (h1, v1) in self.items():
            for (h2, v2) in mass_function.items():
                combined[rule(h1, h2)] += v1 * v2
        return combined

    def _combine_direct_sampling(self, mass_function, rule, sample_count):
        """Helper method for approximatively combining two mass functions using direct sampling."""
        combined = MassFunction()
        samples1 = self.sample(sample_count)
        samples2 = mass_function.sample(sample_count)
        for i in range(sample_count):
            combined[rule(samples1[i], samples2[i])] += 1.0 / sample_count
        return combined

    def _combine_importance_sampling(self, mass_function, sample_count):
        """Helper method for approximatively combining two mass functions using importance sampling."""
        combined = MassFunction()
        for (s1, n) in self.sample(sample_count, as_dict=True).items():
            weight = mass_function.pl(s1)
            for s2 in mass_function.condition(s1).sample(n):
                combined[s2] += weight
        return combined

    def combine_gbt(self, likelihoods, normalization=True, sample_count=None, importance_sampling=True):
        """
        Conjunctively combines the mass function with a mass function obtained from a sequence of
        likelihoods via the generalized Bayesian theorem and returns the combination as a new mass function.

        Equivalent to 'combine_conjunctive(MassFunction.gbt(likelihoods))'.
        By ignoring incompatible likelihoods, it is generally faster than the former
        method and yields a better Monte-Carlo approximation in case of normalization.

        'likelihoods' specifies the conditional plausibilities for a set of singleton hypotheses.
        It can either be a dictionary mapping singleton hypotheses to plausibilities or an iterable
        containing tuples consisting of a singleton hypothesis and a corresponding plausibility value.

        All arguments except for 'likelihoods' must be specified as keyword arguments.
        'normalization' determines whether the resulting mass function is normalized, i.e., whether m({}) == 0.
        If 'sample_count' is not None, the true mass function is approximated using the specified number of samples.
        See 'combine_conjunctive' for details on the effect of setting 'importance_sampling'.
        """
        core = self.core()  # restrict to generally compatible likelihoods
        if isinstance(likelihoods, dict):
            likelihoods = list(likelihoods.items())
        likelihoods = [l for l in likelihoods if l[1] > 0 and l[0] in core]
        if sample_count == None:  # deterministic
            return self.combine_conjunctive(MassFunction.gbt(likelihoods), normalization=normalization)
        else:  # Monte-Carlo
            if not normalization:  # only use importance sampling in case of normalization
                importance_sampling = False
            combined = MassFunction()
            for s, n in self.sample(sample_count, as_dict=True).items():
                if importance_sampling:
                    compatible_likelihoods = [l for l in likelihoods if l[0] in s]
                    weight = 1.0 - reduce(mul, [1.0 - l[1] for l in compatible_likelihoods], 1.0)
                else:
                    compatible_likelihoods = likelihoods
                if not compatible_likelihoods:
                    continue
                if normalization:
                    empty_mass = reduce(mul, [1.0 - l[1] for l in compatible_likelihoods], 1.0)
                for _ in range(n):
                    rv = [random() for _ in range(len(compatible_likelihoods))]
                    subtree_mass = 1.0
                    hyp = set()
                    for k in range(len(compatible_likelihoods)):
                        l = compatible_likelihoods[k][1]
                        norm = 1.0 if hyp or not normalization else 1.0 - empty_mass / subtree_mass
                        if l / norm > rv[k]:
                            hyp.add(compatible_likelihoods[k][0])
                        else:
                            subtree_mass *= 1.0 - l  # only relevant for negative case
                    if importance_sampling:
                        combined[hyp] += weight
                    else:
                        combined[hyp & s] += 1.0
            if normalization:
                return combined.normalize()
            else:
                return combined

    def condition(self, hypothesis, normalization=True):
        """
        Conditions the mass function with 'hypothesis'.

        'normalization' determines whether the resulting conjunctive combination is normalized (must be specified as a keyword argument).

        Shorthand for self.combine_conjunctive(MassFunction({hypothesis:1.0}), normalization).
        """
        m = MassFunction({MassFunction._convert(hypothesis): 1.0})
        return self.combine_conjunctive(m, normalization=normalization)

    def conflict(self, mass_function, sample_count=None):
        """
        Calculates the weight of conflict between two or more mass functions.

        If 'mass_function' is not of type MassFunction, it is assumed to be an iterable containing multiple mass functions.

        The weight of conflict is computed as the (natural) logarithm of the normalization constant in Dempster's rule of combination.
        Returns infinity in case the mass functions are flatly contradicting.
        """
        # compute full conjunctive combination (could be more efficient)
        m = self.combine_conjunctive(mass_function, normalization=False, sample_count=sample_count)
        empty = m[frozenset()]
        m_sum = fsum(m.values())
        diff = m_sum - empty
        if diff == 0.0:
            return float('inf')
        else:
            return -log(diff)

    def normalize(self):
        """
        Normalizes the mass function in-place.

        Sets the mass value of the empty set to 0 and scales all other values such that their sum equals 1.
        For convenience, the method returns 'self'.
        """
        if frozenset() in self:
            del self[frozenset()]
        mass_sum = fsum(self.values())
        if mass_sum != 1.0:
            for (h, v) in self.items():
                self[h] = v / mass_sum
        return self

    def prune(self):
        """
        Removes all non-focal (0 mass) hypotheses in-place.

        For convenience, the method returns 'self'.
        """
        remove = [h for (h, v) in self.items() if v == 0.0]
        for h in remove:
            del self[h]
        return self

    def markov(self, transition_model, sample_count=None):
        """
        Computes the mass function induced by a prior belief (self) and a transition model.

        The transition model expresses a joint belief over the frame of this mass function and a new frame.
        The belief over the frame of this mass function is implicitly assumed to be vacuous.
        The transition model is a function returning the conditional belief over the new frame (as a mass function
        if sample_count=None) while taking a singleton hypothesis of the current frame as input.
        The disjunctive rule of combination is then used to construct the mass function over the new frame.

        If 'sample_count' is not None, the true mass function is approximated using the specified number of samples.
        In this case, 'transition_model' is expected to take a second argument stating how many samples from the corresponding conditional mass function should be returned.
        The return value in this case is expected to be an iterable over sampled hypotheses from the new frame.

        This method can be used to implement the prediction step for estimation in a hidden Markov process (hence the name).
        Under this interpretation, the transition model expresses the mass distribution over successor states given the current state.
        """
        updated = MassFunction()
        if sample_count == None:  # deterministic
            for k, v in self.items():
                predicted = None
                for e in k:
                    if predicted == None:
                        predicted = transition_model(e)
                    else:
                        predicted |= transition_model(e)
                for kp, vp in predicted.items():
                    updated[kp] += v * vp
        else:  # Monte-Carlo
            for s, n in self.sample(sample_count, as_dict=True).items():
                unions = [[] for _ in range(n)]
                for e in s:
                    ts = transition_model(e, n)
                    for i, t in enumerate(ts):
                        unions[i].extend(t)
                for u in unions:
                    updated[u] += 1.0 / sample_count
        return updated

    def map(self, function):
        """
        Maps each hypothesis to a new hypothesis using 'function' and returns the new mass function.

        'function' is a function taking a hypothesis as its only input and returning a new hypothesis
        (i.e., a sequence that can be converted to a 'frozenset').

        Here are some example use cases:

        1. Vacuous extension to a multi-dimensional frame of discernment (m is defined over
        the frame A while the new mass function is defined over the Cartesian product AxB):

            B = {'x', 'y', 'z'}
            m.map(lambda h: itertools.product(h, B))

        2. Projection to a lower dimensional frame (m is defined over AxBxC such that each hypothesis is
        a set of tuples where each tuple consists of 3 elements; the new mass function is defined over BxC):

            m.map(lambda h: (t[1:] for t in h))
        """
        m = MassFunction()
        for (h, v) in self.items():
            m[self._convert(function(h))] += v
        return m

    def pignistic(self):
        """Computes the pignistic transformation and returns it as a new mass function consisting only of singletons."""
        p = MassFunction()
        for (h, v) in self.items():
            if v > 0.0:
                size = float(len(h))
                for s in h:
                    p[(s,)] += v / size
        return p.normalize()

    def local_conflict(self):
        """
        Computes the local conflict measure.

        For more information, see Pal et al. 1993. Uncertainty measures for evidential reasoning II:
        A new measure of total uncertainty. International Journal of Approximate Reasoning.

        Only works for normalized mass functions.
        If the mass function is unnormalized, the method returns float('nan')

        In case the mass function is a probability function (containing only singleton hypotheses),
        it reduces to the classical entropy measure.
        """
        if self[frozenset()] > 0.0:
            return float('nan')
        c = 0.0
        for (h, v) in self.items():
            if v > 0.0:
                c += v * log(len(h) / v, 2)
        return c

    def hartley_measure(self):
        """
        Computes the Hartley-like measure in order to quantify the amount of imprecision.

        For more information, see:
        G. J. Klir (1999), "Uncertainty and information measures for imprecise probabilities: An overview",
        International Symposium on Imprecise Probabilities and Their Applications.
        """
        return fsum([v * log(len(h), 2) for h, v in self.items()])

    def norm(self, m, p=2):
        """
        Computes the p-norm between two mass functions (default is p=2).

        Both mass functions are treated as vectors of mass values.
        """
        d = fsum([(v - m[h]) ** p for (h, v) in self.items()])
        for (h, v) in m.items():
            if h not in self:
                d += v ** p
        return d ** (1.0 / p)

    def is_compatible(self, m):
        """
        Checks whether another mass function is compatible with this one.

        Compatibility means that the mass value of each hypothesis in 'm' is less than
        or equal to the corresponding plausibility given by this mass function.
        """
        return all([self.pl(h) >= v for (h, v) in m.items()])

    def sample(self, n, quantization=True, as_dict=False):
        """
        Returns n random samples from the mass distribution.

        Hypotheses are drawn with a probability proportional to their mass values (with replacement).

        If 'quantization' is True (default), the method performs a quantization of the mass values.
        This means the frequency of a hypothesis h in the sample set is at least int(self[h] * n / t) where t is the sum of all mass values.
        The remaining sample slots (if any) are filled up according to the remainders of the fractions computed in the first step.

        The parameter 'as_dict' determines the type of the returned value.
        If 'as_dict' is False (default), a list of length n is returned.
        Otherwise, the result is a dictionary specifying the number of samples for each hypothesis.
        """
        if not isinstance(n, int):
            raise TypeError("n must be int")
        samples = {h: 0 for h in self} if as_dict else []
        mass_sum = fsum(self.values())
        if quantization:
            remainders = []
            remaining_sample_count = n
            for (h, v) in self.items():
                fraction = n * v / mass_sum
                quotient = int(fraction)
                if quotient > 0:
                    if as_dict:
                        samples[h] = quotient
                    else:
                        samples.extend([h] * quotient)
                remainders.append((h, fraction - quotient))
                remaining_sample_count -= quotient
            remainders.sort(reverse=True, key=lambda hv: hv[1])
            for h, _ in remainders[:remaining_sample_count]:
                if as_dict:
                    samples[h] += 1
                else:
                    samples.append(h)
        else:
            rv = [uniform(0.0, mass_sum) for _ in range(n)]
            hypotheses = sorted(self.items(), reverse=True, key=lambda hv: hv[1])
            for i in range(n):
                mass = 0.0
                for (h, v) in hypotheses:
                    mass += v
                    if mass >= rv[i]:
                        if as_dict:
                            samples[h] += 1
                        else:
                            samples.append(h)
                        break
        if not as_dict:
            shuffle(samples)
        return samples

    def is_probabilistic(self):
        """
        Checks whether the mass function is a probability function.

        Returns True if and only if all hypotheses are singletons (normalization is ignored).
        """
        return all([len(h) == 1 for h in self.keys()])

    def sample_probability_distributions(self, n):
        """
        Randomly generates n compatible probability distributions from the mass function.

        The result is a list of n independently sampled probability distributions expressed as mass functions.
        This can be useful for estimating various statistical measures like the minimum or maximum entropy consistent with the mass distribution.
        """
        samples = [MassFunction() for _ in range(n)]
        for i in range(n):
            for (h, v) in self.items():
                if len(h) == 1:
                    samples[i][h] += v
                else:
                    rv = [random() for _ in range(len(h))]
                    total = fsum(rv)
                    for k, s in enumerate(h):
                        samples[i][{s}] += rv[k] * v / total
        return samples

    def max_bel(self):
        """
        Returns the singleton with the highest belief.

        In case there are multiple singletons with maximum belief, only one of them is returned.
        Returns None, if the mass function does not contain any hypotheses.
        """
        return self._max_singleton(self.bel)

    def max_pl(self):
        """
        Returns the singleton with the highest plausibility.

        In case there are multiple singletons with maximum plausibility, only one of them is returned.
        Returns None, if the mass function does not contain any hypotheses.
        """
        return self._max_singleton(self.pl)

    def _max_singleton(self, f):
        st = self.singletons()
        if st:
            value_list = [(f(s), s) for s in st]
            shuffle(value_list)
            return max(value_list)[1]
        else:
            return None

    def to_dict(self):
        """Convert a mass function only consisting of singletons to a dictionary by removing each enclosing frozenset."""
        if not self.is_probabilistic():
            raise Exception('mass function must only contain singletons')
        return {tuple(h)[0]: v for h, v in self.items()}

    @staticmethod
    def from_dict(d):
        """Convert a dictionary to a mass function by enclosing each key with a frozenset."""
        if isinstance(d, MassFunction):
            return d
        else:
            return MassFunction({frozenset((h,)): v for h, v in d.items()})

    @staticmethod
    def from_possibility(poss):
        """
        Constructs a consonant mass function from a possibility distribution.

        For more information, see:
        D. Dubois, H. Prade (1982), "On several representations of an uncertainty body of evidence",
        Fuzzy Information and Decision Processes, 167-181.
        """
        if isinstance(poss, MassFunction):
            poss = poss.to_dict()  # remove enclosing sets
        H, P = zip(
            *sorted(poss.items(), key=lambda e: e[1], reverse=True))  # sort possibility values in descending order
        m = MassFunction()
        m[H] = P[-1]
        for i in range(len(H) - 1):
            m[H[:i + 1]] = P[i] - P[i + 1]
        return m

    @staticmethod
    def pignistic_inverse(p):
        """
        Constructs a consonant mass function from a pignistic probability distribution by applying the inverse pignistic transformation.

        For more information, see:
        D. Dubois, H. Prade, P. Smets (2008), "A definition of subjective possibility",
        International Journal of Approximate Reasoning 48 (2), 352-364.
        """
        p = MassFunction.from_dict(p)
        poss = MassFunction({h1: fsum([min(p[h1], p[h2]) for h2 in p.keys()]) for h1 in p.keys()})
        return MassFunction.from_possibility(poss)

    @staticmethod
    def _to_array_index(hypothesis, frame):
        """Map a hypothesis to an array index given a frame of discernment."""
        index = 0
        for i, s in enumerate(frame):
            if s in hypothesis:
                index += 2 ** i
        return index

    @staticmethod
    def _from_array_index(index, frame):
        """Map an array index to a hypothesis given a frame of discernment."""
        hypothesis = set()
        for i, s in enumerate(frame):
            if 2 ** i & index:
                hypothesis.add(s)
        return frozenset(hypothesis)

    def to_array(self, frame):
        """
        Convert the mass function to a NumPy array.

        Hypotheses are mapped to array indices using '_to_array_index'.
        The resulting array has 2^n entries where n is the size of the frame of discernment.
        """
        a = numpy.zeros(2 ** len(frame))
        for h, v in self.items():
            a[MassFunction._to_array_index(h, frame)] = v
        return a

    @staticmethod
    def from_array(a, frame):
        """
        Convert a NumPy array to a mass function given a frame of discernment.

        Array indices are mapped to hypotheses using '_from_array_index'.
        """
        m = MassFunction()
        for i, v in enumerate(a):
            if v > 0.0:
                m[MassFunction._from_array_index(i, frame)] = v
        return m

    @staticmethod
    def _confidence_intervals(histogram, alpha):
        """Compute Goodman confidence intervals."""
        p_lower = {}
        p_upper = {}
        a = chi2.ppf(1. - alpha / len(histogram), 1)
        n = float(sum(histogram.values()))
        for h, n_h in histogram.items():
            delta_h = a * (a + 4. * n_h * (n - n_h) / n)
            p_lower[h] = (a + 2. * n_h - sqrt(delta_h)) / (2. * (n + a))
            p_upper[h] = (a + 2. * n_h + sqrt(delta_h)) / (2. * (n + a))
        return p_lower, p_upper

    @staticmethod
    def from_samples(histogram, method='idm', alpha=0.05, s=1.0):
        """
        Generate a mass function from an empirical probability distribution that was obtained from a limited number of samples.
        This makes the expected deviation of the empirical distribution from the true distribution explicit.

        'histogram' represents the empirical distribution. It is a dictionary mapping each possible event to the respective
        number of observations (represented as integers).

        'method' determines the algorithm used for generating the mass function.
        Except for method 'bayesian', all algorithms are based on the idea that the true probabilities lie within confidence intervals
        represented by the mass function with confidence level 1 - 'alpha'.

        The following modes are supported:

        'idm': Imprecise Dirichlet model. A small amount of mass (controlled by 's') is assigned to the entire frame.
        For more information on 'idm', see:
        P. Walley (1996), "Inferences from multinomial data: learning about a bag of marbles",
        Journal of the Royal Statistical Society. Series B (Methodological), 3-57.

        'maxbel': Maximize the total belief by solving a linear program. (Attention: this becomes very computationally expensive
        for larger numbers of events.)

        'maxbel-ordered': Similar to 'maxbel' except that the events are assumed to have a natural order (e.g., intervals), in which case
        the mass function can be computed analytically and thus much faster.

        For more information on 'maxbel' and 'maxbel-ordered', see:
        T. Denoeux (2006), "Constructing belief functions from sample data using multinomial confidence regions",
        International Journal of Approximate Reasoning 42, 228-252.

        'mcd': Compute the least committed consonant mass function whose pignistic transformation lies within the confidence interval
        induced by 'alpha'. Like 'maxbel', it is based on solving a linear program and quickly becomes computationally expensive.

        'mcd-approximate': An approximation of 'mcd' that can be computed much more efficiently.

        For more information on these two methods, see:
        A. Aregui, T. Denoeux (2008), "Constructing consonant belief functions from sample data using confidence sets of pignistic probabilities",
        International Journal of Approximate Reasoning 49, 575-594.

        'bayesian': Construct a Bayesian mass function based on the relative frequencies. In addition, additive smoothing is applied (controlled by 's').

        In case the sample number is 0, returns a vacuous mass function (or uniform distribution for 'bayesian').

        (Requires SciPy for computing confidence intervals and solving linear programs.)
        """
        if not isinstance(histogram, dict):
            raise TypeError('histogram must be of type dict')
        for v in histogram.values():
            if not isinstance(v, int):
                raise TypeError('all histogram values must be of type int')
        if not histogram:
            return MassFunction()
        if sum(histogram.values()) == 0:  # return vacuous/uniform belief if there are no samples
            vac = MassFunction({tuple(histogram.keys()): 1})
            if method == 'bayesian':
                return vac.pignistic()
            else:
                return vac
        if method == 'bayesian':
            return MassFunction({(h,): v + s for h, v in histogram.items()}).normalize()
        elif method == 'idm':
            return MassFunction._from_samples_idm(histogram, s)
        elif method == 'maxbel':
            return MassFunction._from_samples_maxbel(histogram, alpha)
        elif method == 'maxbel-ordered':
            return MassFunction._from_samples_maxbel(histogram, alpha, ordered=True)
        elif method == 'mcd':
            return MassFunction._from_samples_mcd(histogram, alpha)
        elif method == 'mcd-approximate':
            return MassFunction._from_samples_mcd(histogram, alpha, approximate=True)
        raise ValueError('unknown method: %s' % method)

    @staticmethod
    def _from_samples_idm(histogram, s):
        """
        Reference:
        P. Walley (1996), "Inferences from multinomial data: learning about a bag of marbles",
        Journal of the Royal Statistical Society. Series B (Methodological), 3-57.
        """
        total = sum(histogram.values())
        m = MassFunction()
        for h, c in histogram.items():
            m[(h,)] = float(c) / (total + s)
        m[MassFunction._convert(histogram.keys())] = float(s) / (total + s)
        return m

    @staticmethod
    def _from_samples_maxbel(histogram, alpha, ordered=False):
        """
        Reference:
        T. Denoeux (2006), "Constructing belief functions from sample data using multinomial confidence regions",
        International Journal of Approximate Reasoning 42, 228-252.
        """
        p_lower, p_upper = MassFunction._confidence_intervals(histogram, alpha)

        def p_lower_set(hs):
            l = u = 0
            for h in H:
                if h in hs:
                    l += p_lower[h]
                else:
                    u += p_upper[h]
            return max(l, 1 - u)

        if ordered:
            H = sorted(histogram.keys())
            m = MassFunction()
            for i1, h1 in enumerate(H):
                m[(h1,)] = p_lower[h1]
                for i2, h2 in enumerate(H[i1 + 1:]):
                    i2 += i1 + 1
                    if i2 == i1 + 1:
                        v = p_lower_set(H[i1:i2 + 1]) - p_lower[h1] - p_lower[h2]
                    else:
                        v = p_lower_set(H[i1:i2 + 1]) - p_lower_set(H[i1 + 1:i2 + 1]) - p_lower_set(
                            H[i1:i2]) + p_lower_set(H[i1 + 1:i2])
                    if v > 0:
                        m[H[i1:i2 + 1]] = v
            return m
        else:
            H = list(histogram.keys())
            L = 2 ** len(H)
            initial = numpy.zeros(L)
            cons = []
            singletons = lambda index: [i for i in range(len(H)) if 2 ** i & index]
            # constraint (24)
            bel = lambda index, m: fsum(m[sum([2 ** i for i in h_ind])] for h_ind in powerset(singletons(index)))
            c24 = lambda m, i: p_lower_set(MassFunction._from_array_index(i, H)) - bel(i, m)
            for i in range(L):
                cons.append(partial(c24, i=i))
            # constraint (25)
            cons.append(lambda m: m.sum() - 1.0)
            cons.append(lambda m: 1.0 - m.sum())
            # constraint (26)
            for i in range(L):
                cons.append(partial(lambda m, i_s: m[i_s], i_s=i))
            f = lambda m: -1 * 2 ** len(H) * fsum([m[i] * 2 ** (-len(singletons(i))) for i in range(L)])
            m_optimal = fmin_cobyla(f, initial, cons, disp=0)
            return MassFunction.from_array(m_optimal, H)

    @staticmethod
    def _from_samples_mcd(histogram, alpha, approximate=False):
        """
        Reference:
        A. Aregui, T. Denoeux (2008), "Constructing consonant belief functions from sample data using confidence
        sets of pignistic probabilities", International Journal of Approximate Reasoning 49, 575-594.
        """
        p_lower, p_upper = MassFunction._confidence_intervals(histogram, alpha)
        H = list(histogram.keys())
        if approximate:
            # approximate possibility distribution
            poss = {h1: min(1, fsum([min(p_upper[h1], p_upper[h2]) for h2 in H])) for h1 in H}
        else:
            # optimal possibility distribution (based on linear programming)
            poss = {h: 0. for h in H}
            for k, h_k in enumerate(H):
                S_k = {l for l in range(len(H)) if p_lower[H[l]] >= p_upper[h_k]}
                S_k.add(k)
                I_k = {l for l in range(len(H)) if p_upper[H[l]] < p_lower[h_k]}
                P_k = set(range(len(H))).difference(S_k.union(I_k))
                for A in powerset(P_k):
                    G = S_k.union(A)
                    G_c = set(range(len(H))).difference(G)
                    cons = []
                    # constraint (26)
                    for i, h in enumerate(H):
                        cons.append(partial(lambda p, i_s, p_s: p[i_s] - p_s, i_s=i, p_s=p_lower[h]))  # lower bound
                        cons.append(partial(lambda p, i_s, p_s: p_s - p[i_s], i_s=i, p_s=p_upper[h]))  # upper bound
                    # constraint (27)
                    cons.append(lambda p: 1. - sum(p))
                    cons.append(lambda p: sum(p) - 1.)
                    # constraint (30)
                    for i in G:
                        cons.append(partial(lambda p, i_s: p[i_s] - p[k], i_s=i))
                    # constraint (31)
                    for i in G_c:
                        cons.append(partial(lambda p, i_s: p[k] - p[i_s], i_s=i))
                    initial = [1.0 / len(H)] * len(H)
                    f = lambda p: -(fsum([p[i] for i in G_c]) + len(G) * p[k])
                    poss_optimal = fmin_cobyla(f, initial, cons, disp=0)
                    poss[h_k] = max(poss[h_k], -f(poss_optimal))
        return MassFunction.from_possibility(poss)


def powerset(iterable):
    """
    Returns an iterator over the power set of 'set'.

    'set' is an arbitrary iterator over hashable elements.
    All returned subsets are of type 'frozenset'.
    """
    return map(frozenset, chain.from_iterable(combinations(iterable, r) for r in range(len(iterable) + 1)))


def gbt_m(hypothesis, likelihoods, normalization=True):
    """
    Computes the mass value of 'hypothesis' using the generalized Bayesian theorem.

    Equivalent to MassFunction.gbt(likelihoods, normalization)[hypothesis].
    """
    if isinstance(likelihoods, dict):
        likelihoods = list(likelihoods.items())
    q = gbt_q(hypothesis, likelihoods, normalization)
    return q * reduce(mul, [1.0 - l[1] for l in likelihoods if l[0] not in hypothesis], 1.0)


def gbt_bel(hypothesis, likelihoods, normalization=True):
    """
    Computes the belief of 'hypothesis' using the generalized Bayesian theorem.

    Equivalent to MassFunction.gbt(likelihoods, normalization).bel(hypothesis).
    """
    if isinstance(likelihoods, dict):
        likelihoods = list(likelihoods.items())
    eta = _gbt_normalization(likelihoods) if normalization else 1.0
    exc = reduce(mul, [1.0 - l[1] for l in likelihoods if l[0] not in hypothesis], 1.0)
    all_hyp = reduce(mul, [1.0 - l[1] for l in likelihoods], 1.0)
    return eta * (exc - all_hyp)


def gbt_pl(hypothesis, likelihoods, normalization=True):
    """
    Computes the plausibility of 'hypothesis' using the generalized Bayesian theorem.

    Equivalent to MassFunction.gbt(likelihoods, normalization).pl(hypothesis).
    """
    if isinstance(likelihoods, dict):
        likelihoods = list(likelihoods.items())
    eta = _gbt_normalization(likelihoods) if normalization else 1.0
    return eta * (1.0 - reduce(mul, [1.0 - l[1] for l in likelihoods if l[0] in hypothesis], 1.0))


def gbt_q(hypothesis, likelihoods, normalization=True):
    """
    Computes the commonality of 'hypothesis' using the generalized Bayesian theorem.

    Equivalent to MassFunction.gbt(likelihoods, normalization).q(hypothesis).
    """
    if isinstance(likelihoods, dict):
        likelihoods = list(likelihoods.items())
    eta = _gbt_normalization(likelihoods) if normalization else 1.0
    return eta * reduce(mul, [l[1] for l in likelihoods if l[0] in hypothesis], 1.0)


def gbt_pignistic(singleton, likelihoods):
    """
    Computes the pignistic probability of 'singleton' for the belief function obtained
    by applying the generalized Bayesian theorem to 'likelihoods'.

    This function has time complexity O(len(likelihoods)**2) and is equivalent to the following
    expression (which has exponential complexity):
    MassFunction.gbt(likelihoods).pignistic()[(singleton,)]
    """
    if isinstance(likelihoods, dict):
        likelihoods = list(likelihoods.items())
    singleton_lh = None
    lh_values = []
    for h, v in likelihoods:
        if h == singleton:
            singleton_lh = v
        else:
            lh_values.append(v)
    if singleton_lh is None:
        raise ValueError('singleton %s is not contained in likelihoods' % repr(singleton))
    m_sum = _gbt_pignistic_recursive(lh_values, 0)
    eta = _gbt_normalization(likelihoods)
    return sum([eta * v * singleton_lh / (c + 1.) for c, v in enumerate(m_sum)])


def _gbt_pignistic_recursive(likelihoods, i):
    """
    Helper function for recursively computing the pignistic probability corresponding to the GBT.

    This function computes the sum over all mass values (obtained via the GBT) and groups them by the
    cardinalities of the corresponding sets.
    """
    if i == len(likelihoods) - 1:
        m_sum = [0.] * (len(likelihoods) + 1)
        m_sum[0] = 1. - likelihoods[i]
        m_sum[1] = likelihoods[i]
        return m_sum
    else:
        m_sum = _gbt_pignistic_recursive(likelihoods, i + 1)
        m_sum_inc = [0.] * (len(likelihoods) + 1)
        m_sum_exc = [0.] * (len(likelihoods) + 1)
        for k in range(len(likelihoods) + 1):
            if k < len(likelihoods):
                m_sum_inc[k + 1] = m_sum[k] * likelihoods[i]
            m_sum_exc[k] = m_sum[k] * (1. - likelihoods[i])
            m_sum[k] = m_sum_inc[k] + m_sum_exc[k]
        return m_sum


def _gbt_normalization(likelihoods):
    """Helper function for computing the GBT normalization constant."""
    return 1.0 / (1.0 - reduce(mul, [1.0 - l[1] for l in likelihoods], 1.0))

# Env

In [4]:
from itertools import chain, combinations
import torch

from ontolearn.knowledge_base import KnowledgeBase

from ontolearn.utils.static_funcs import concept_len
from ontolearn.metrics import F1
from ontolearn.quality_funcs import evaluate_concept
from ontolearn.learning_problem import PosNegLPStandard
from owlapy.owl_individual import OWLNamedIndividual
from owlapy import owl_expression_to_dl
import json
from ontolearn.heuristics import CELOEHeuristic
from ontolearn.learners import CELOE

import numpy as np

from ontolearn.refinement_operators import ModifiedCELOERefinement

from collections import defaultdict

import pandas as pd
import time
from owlapy.class_expression import OWLDataSomeValuesFrom

from owlapy.owl_literal import OWLLiteral

from owlapy.owl_datatype import OWLDatatype
from owlapy.iri import IRI
from owlapy.class_expression import OWLFacetRestriction, OWLFacet, OWLDatatypeRestriction

def nugyen_entropy(m):
    all_hypothesis = []
    for i in m:
        all_hypothesis.append(m[i] * np.log2(m[i]))
    return np.sum(all_hypothesis)
    
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer

def pos_and_neg_parser(lp_path):
    pos_file = lp_path + r'/pos.txt'
    neg_file = lp_path + r'/neg.txt'

    with open(pos_file, encoding='utf-8') as file:
        pos_data = file.readlines()

    with open(neg_file, encoding='utf-8') as file:
        neg_data = file.readlines()

    pos = set([OWLNamedIndividual(x) for x in [x.replace('\n', '') for x in pos_data][:-1]])
    neg = set([OWLNamedIndividual(x) for x in [x.replace('\n', '') for x in neg_data]])
    return pos, neg


def experiment_report(lp_name, models, n_trials, original_g, pos, neg):


    kb = KnowledgeBase(path=original_g)

    lp = PosNegLPStandard(pos=pos, neg=neg)
    lp.all = pos.union(neg)

    results_dict = defaultdict(list)

    for m in models:
        for i in range(n_trials):
            start_time = time.time()
            if 'CLIP' == m.name:
                model = m(knowledge_base=kb, refinement_operator=ModifiedCELOERefinement(kb),
                             predictor_name='SetTransformer').get_length_predictor()
            elif 'celoe_python' == m.name:
                qual = F1()
                heur = CELOEHeuristic(expansionPenaltyFactor=0.05, startNodeBonus=1.0, nodeRefinementPenalty=0.01)
                op = ModifiedCELOERefinement(knowledge_base=kb, use_negation=False, use_all_constructor=False)
                model = CELOE(knowledge_base=kb,
                              max_runtime=600,
                              refinement_operator=op,
                              quality_func=qual,
                              heuristic_func=heur,
                              max_num_of_concepts_tested=100,
                              iter_bound=100)

            elif 'Lyra' == m.name:
                model = m(original_g,pos, neg)
            else:
                model = m(kb)

            if 'Lyra' == m.name:
                model.fit()
            else:
                model.fit(lp)

            end_time = time.time()
            if model.name != 'Lyra':
                f1 = evaluate_concept(kb, model.best_hypotheses(), F1(), lp.encode_kb(kb)).q
                con_len = concept_len(model.best_hypotheses())
                expr = owl_expression_to_dl(model.best_hypotheses())
                results_dict[model.name].append({'trial': i,
                                             'f1': f1,
                                             'time': end_time - start_time,
                                             'concept_len': con_len,
                                             'DL': expr})
            else:
                f1 = evaluate_concept(kb, model.best_hypotheses()[0], F1(), lp.encode_kb(kb)).q
                con_len = concept_len(model.best_hypotheses()[0])
                expr = owl_expression_to_dl(model.best_hypotheses()[0])
                results_dict[model.name].append({'trial': i,
                                             'f1': f1,
                                             'time': end_time - start_time,
                                             'concept_len': con_len,
                                             'DL': expr})
            
            print('trial: ', i, 'model name: ', model.name, 'f1: ', f1, 'time: ', np.round(end_time - start_time, 3),
                  'concept_len: ', con_len, 'expression: ', expr)
            print('=========================================================================')
            if model.name == 'Lyra':
                best_hypo = model.best_hypotheses()
            else:
                best_hypo = list(set(model.best_hypotheses(n = 5)))
            for i in best_hypo:
                print('f1: ', evaluate_concept(kb, i, F1(), lp.encode_kb(kb)).q, '/n',
                      'exp:', owl_expression_to_dl(i))

    report_dict = {}
    for k, v in results_dict.items():
        f1 = []
        len_ = []
        time_ = []
        for i in v:
            f1.append(i['f1'])
            len_.append(i['concept_len'])
            time_.append(i['time'])
        report_dict[k] = {'models': ['EvoLearner','Lyra', 'Drill'],
                          'f1_mean': np.round(np.array(f1).mean(), 5),
                          'f1_std': np.round(np.array(f1).std(), 5),
                          'time': np.round(np.array(time_).mean(), 5),
                          'time_std': np.round(np.array(time_).std(), 5),
                          'DL_len': np.array(len_).mean()}
    report_df = pd.DataFrame(report_dict).T
    report_df.to_csv(f"{lp_name}_out.csv", index=False)

    with open(f"{lp_name}_exp_results.csv", 'w', encoding="utf-8") as fp:
        json.dump(results_dict, fp)


def create_numeric_data_some_value(datatype, value, resteriction):
    double_datatype = OWLDatatype(IRI.create('http://www.w3.org/2001/XMLSchema#double'))
    literal = OWLLiteral(value)
    if resteriction == 'min_inc':
        facet_restriction = OWLFacetRestriction(OWLFacet.MIN_INCLUSIVE, literal)

    if resteriction == 'max_inc':
        facet_restriction = OWLFacetRestriction(OWLFacet.MAX_INCLUSIVE, literal)

    datatype_restriction = OWLDatatypeRestriction(double_datatype, [facet_restriction])

    d = OWLDataSomeValuesFrom(datatype, datatype_restriction)

    return d

def check_generators(iterable):
    try:
        next(iterable)
    except StopIteration:
        return False
    return

In [5]:
# from ontolearn.knowledge_base import KnowledgeBase
# from ontolearn.metrics import F1, Accuracy
# from ontolearn.quality_funcs import evaluate_concept
# from ontolearn.learning_problem import PosNegLPStandard
# from owlapy import owl_expression_to_dl
from gymnasium import spaces
from scipy.special import softmax
import functools
from copy import copy

import random
# import pandas as pd
# from collections import defaultdict

from catboost import CatBoostClassifier, Pool
# import numpy as np
# import json
import tempfile
import os

# from belief_functions.BFT import *
# from encoders.expression_encoders import ExpressionEncoder
# from utils.utils import *
from owlapy.class_expression import OWLDataHasValue

from owlapy.owl_literal import OWLLiteral
from owlapy.owl_property import OWLDataPropertyExpression
from pettingzoo import ParallelEnv


class Concept_Dec_POMPD(ParallelEnv):
    metadata = {
        "name": "concept_dec_pompd_v0", "render_modes": ["human"]
    }

    def __init__(
            self,
            file_path,
            pos,
            neg,
            reward_type,
            max_action,
            render=False,
            render_mode=None,
            max_card=10,
            device='cpu'
    ):
        self.cntr = 0
        self.device = device
        self.filepath = file_path
        self.kb = KnowledgeBase(path=self.filepath)
        self.render_mode = render_mode
        self.cg = self.kb.generator
        self.pos = pos
        self.neg = neg
        self.f1_score = 0

        lp = PosNegLPStandard(pos=pos, neg=neg)
        lp.all = pos.union(neg)
        self.lp = lp

        self.reward_type = reward_type
        self.max_action = max_action

        self.set_operators = [
            self.cg.union,
            self.cg.intersection,
            self.cg.negation,
        ]
        self.quantifiers = [
            self.cg.existential_restriction,
            self.cg.universal_restriction,
            self.cg.has_value_restriction
        ]
        self.cardinality = [
            self.cg.min_cardinality_restriction,
            self.cg.max_cardinality_restriction,
            self.cg.exact_cardinality_restriction
        ]
        self.numeric_dp = self._get_numeric_splits()

        self.concepts = list(self.kb.concepts)
        self.properties = list(self.kb.object_properties) + list(self.kb.data_properties)

        self.mappings = {
            "concepts": dict(
                [(idx, val) for idx, val in enumerate(self.concepts)]
            ),
            "properties": dict(
                [(idx, val) for idx, val in enumerate(self.properties)]
            ),
            "set": dict(
                [(idx, val) for idx, val in enumerate(self.set_operators)]
            ),
            "quantifier": dict(
                [(idx, val) for idx, val in enumerate(self.quantifiers)]
            ),
            "cardinality": dict(
                [(idx, val) for idx, val in enumerate(self.cardinality)]
            )
        }
        self.max_card = max_card
        self.possible_agents = ["a_0", "a_1"]
        self.object_properties_ = list(self.kb.object_properties)

        self.expression_encoder = ExpressionEncoder(device=device)

        self.data_obj_splits = self._get_split_catboost()
        self.boolean_dp = {}  # TODO

    def reset(self, seed=None, options=None):
        self.expression = self.init_expression()
        self.agents = copy(self.possible_agents)
        self.infos = {}
        self.cntr = 0
        self.reward = 0
        self.step_reward = 0
        self.nugyen_entropy = 0

        observations = {a: self._get_observation_state() for a in self.agents}

        infos = {a: {} for a in self.agents}

        return observations['a_0'], infos

    def get_theta(self, actions):
        return list(powerset(actions))[1:]

    def get_bpa(self, bpa):
        return softmax(bpa).flatten()

    def _get_masses(self, actions, bpa):

        theta = self.get_theta(actions)
        bpa = self.get_bpa(bpa)

        m1 = {}
        for i in range(len(theta)):
            m1[theta[i]] = bpa[i]
        return MassFunction(m1)

    def _get_best_action(self, m1, m2, singletons=True):
        combined = m1 & m2
        if singletons:
            best = 0.0
            best_action = frozenset({})
            for c in combined.all():
                if combined[c] > best:
                    best = combined[c]
                    best_action = c
            return list(best_action), nugyen_entropy(combined)
        else:
            return list(max(combined.pignistic(), key=combined.pignistic().get)), nugyen_entropy(combined)

    def _extract_actions(self, agent):
        c1 = agent["actions"][:2]
        c2 = agent["actions"][2:4]
        p = agent["actions"][4:6]
        bo = agent["actions"][6:8]
        q = agent["actions"][8:10]
        eq = agent["actions"][10:12]
        flags = agent["actions"][12:14]
        stop = agent["actions"][14:16]
        card = agent["actions"][16:]

        return c1, c2, p, bo, q, eq, flags, stop, card

    def _extract_bpa(self, agent):
        c1 = agent["masses"][0]
        c2 = agent["masses"][1]
        p = agent["masses"][2]
        bo = agent["masses"][3]
        q = agent["masses"][4]
        e = agent["masses"][5]
        flags = agent["masses"][6]
        stop = agent["masses"][7]
        card = agent["masses"][8]
        return c1, c2, p, bo, q, e, flags, stop, card

    def _get_observation_state(self):
        return self.expression_encoder.encode(str(self.expression))

    def step(self, actions):
        self.cntr += 1
        info = {}
        old_expression = self.expression

        c1_a1, c2_a1, p_a1, bo_a1, q_a1, e_a1, flags_a1, stop_a1, card_a1 = self._extract_actions(
            actions["a_0"]
        )
        c1_a2, c2_a2, p_a2, bo_a2, q_a2, e_a2, flags_a2, stop_a2, card_a2 = self._extract_actions(
            actions["a_1"]
        )

        bpa_a1_c1, bpa_a1_c2, bpa_a1_p, bpa_a1_bo, bpa_a1_q, bpa_a1_e, bpa_a1_flags, bpa_a1_stop, bpa_a1_card = (
            self._extract_bpa(actions["a_0"])
        )
        bpa_a2_c1, bpa_a2_c2, bpa_a2_p, bpa_a2_bo, bpa_a2_q, bpa_a2_e, bpa_a2_flags, bpa_a2_stop, bpa_a2_card = (
            self._extract_bpa(actions["a_1"])
        )

        c1_a1, c1_a2 = self.constrained_action_sample(c1_a1, c1_a2)

        c2_a1, c2_a2 = self.constrained_action_sample(c2_a1, c2_a2)

        p_a1, p_a2 = self.constrained_action_sample(p_a1, p_a2)

        bo_a1, bo_a2 = self.constrained_action_sample(bo_a1, bo_a2)

        q_a1, q_a2 = self.constrained_action_sample(q_a1, q_a2)

        e_a1, e_a2 = self.constrained_action_sample(e_a1, e_a2)

        flags_a1, flags_a2 = self.constrained_action_sample(flags_a1, flags_a2)

        stop_a1, stop_a2 = self.constrained_action_sample(stop_a1, stop_a2)

        card_a1, card_a2 = self.constrained_action_sample(card_a1, card_a2)

        c1_m1 = self._get_masses(c1_a1, bpa_a1_c1)
        c1_m2 = self._get_masses(c1_a2, bpa_a2_c1)
        best_action_1, a1_entropy = self._get_best_action(c1_m1, c1_m2)

        c2_m1 = self._get_masses(c2_a1, bpa_a1_c2)
        c2_m2 = self._get_masses(c2_a2, bpa_a2_c2)
        best_action_2, a2_entropy = self._get_best_action(c2_m1, c2_m2)

        p_m1 = self._get_masses(p_a1, bpa_a1_p)
        p_m2 = self._get_masses(p_a2, bpa_a2_p)
        best_action_3, a3_entropy = self._get_best_action(p_m1, p_m2)

        bo_m1 = self._get_masses(bo_a1, bpa_a1_bo)
        bo_m2 = self._get_masses(bo_a2, bpa_a2_bo)
        best_action_4, a4_entropy = self._get_best_action(bo_m1, bo_m2)

        q_m1 = self._get_masses(q_a1, bpa_a1_q)
        q_m2 = self._get_masses(q_a2, bpa_a2_q)
        best_action_5, a5_entropy = self._get_best_action(q_m1, q_m2)

        flags_m1 = self._get_masses(flags_a1, bpa_a1_flags)
        flags_m2 = self._get_masses(flags_a2, bpa_a2_flags)
        best_action_6, a6_entropy = self._get_best_action(flags_m1, flags_m2)

        stop_m1 = self._get_masses(stop_a1, bpa_a1_stop)
        stop_m2 = self._get_masses(stop_a2, bpa_a2_stop)
        best_action_7, a7_entropy = self._get_best_action(stop_m1, stop_m2)

        e_m1 = self._get_masses(e_a1, bpa_a1_e)
        e_m2 = self._get_masses(e_a2, bpa_a2_e)
        best_action_8, a8_entropy = self._get_best_action(e_m1, e_m2)

        card_m1 = self._get_masses(card_a1, bpa_a1_card)
        card_m2 = self._get_masses(card_a2, bpa_a2_card)
        best_action_9, a9_entropy = self._get_best_action(card_m1, card_m2)
        self.nugyen_entropy = - np.sum([a1_entropy, a2_entropy, a3_entropy, a4_entropy, a5_entropy, a6_entropy,
                                     a7_entropy, a8_entropy, a9_entropy])
        mass_functions = [
            c1_m1, c1_m2,
            c2_m1, c2_m2,
            p_m1, p_m2,
            bo_m1, bo_m2,
            q_m1, q_m2,
            flags_m1, flags_m2,
            stop_m1, stop_m2,
            e_m1, e_m2,
            card_m1, card_m2]

        pl_bel_differences = []
        for m in mass_functions:
            pl_bel_differences.append((np.array(list(m.pl().values())) - np.array(list(m.bel().values()))).mean())

        self.pl_bel_gaps = np.mean(pl_bel_differences)

        self.conflict = np.mean([c1_m1.conflict(c1_m2),
                                 c2_m1.conflict(c2_m2),
                                 p_m1.conflict(p_m2),
                                 bo_m1.conflict(bo_m2),
                                 q_m1.conflict(q_m2),
                                 flags_m1.conflict(flags_m2),
                                 stop_m1.conflict(stop_m2),
                                 e_m1.conflict(e_m2),
                                 card_m1.conflict(card_m2)])
        info['pl_bel_gaps'] = self.pl_bel_gaps
        info['conflict'] = self.conflict

        if (self.cntr == self.max_action) or (best_action_7[0] == 1):
            stop = True
        else:
            stop = False
        old_f1_score = evaluate_concept(
            self.kb, self.expression, F1(), self.lp.encode_kb(self.kb)
        ).q
        if stop:
            info["f1"] = old_f1_score
            terminations = truncations = {"a_0": True, "a_1": True}
            self.reset()
            # rewards = {"a_0": 0, "a_1": 0}

        if not stop:

            terminations = truncations = {"a_0": False, "a_1": False}
            set_operators = self.mappings["set"][best_action_4[0]]
            if best_action_6[0] == 0:
                if best_action_4[0] == 2:
                    self.expression = set_operators(self.expression)
                else:
                    self.expression = set_operators(
                        (
                            self.expression,
                            self.mappings["concepts"][best_action_1[0]],
                        )
                    )
            else:
                quantifier = self.mappings["quantifier"][best_action_5[0]]
                cardinality = self.mappings['cardinality'][best_action_8[0]]
                if self.expression == self.cg.thing:
                    if not isinstance(self.mappings["properties"][best_action_3[0]], OWLDataPropertyExpression):
                        if best_action_6[0] == 1:
                            self.expression = quantifier(
                                self.mappings["concepts"][best_action_1[0]],
                                self.mappings["properties"][best_action_3[0]],
                            )
                        if best_action_6[0] == 2:
                            self.expression = cardinality(
                                self.mappings["concepts"][best_action_1[0]],
                                self.mappings["properties"][best_action_3[0]],
                                best_action_9[0]
                            )
                    else:
                        if self.mappings["properties"][
                            best_action_3[0]] in self.boolean_dp:  # TODO: create booldp dictionary
                            value = list(self.kb.get_data_property_values(self.pos_example,
                                                                          self.boolean_dp[self.mappings["properties"][
                                                                              best_action_3[0].reminder]]))[0]
                            if (str(value).lower() == 'false') or (str(value).lower() == '0'):
                                data_has_value = OWLDataHasValue(self.mappings["properties"][best_action_3[0]],
                                                                 OWLLiteral(False))  # HEREEE
                            else:
                                data_has_value = OWLDataHasValue(self.mappings["properties"][best_action_3[0]],
                                                                 OWLLiteral(True))
                            if best_action_4[0] != 2:
                                self.expression = set_operators((self.expression, data_has_value))
                            else:
                                self.expression = set_operators(self.cg.union(self.expression, data_has_value))
                        else:
                            try:
                                if self.mappings["properties"][best_action_3[0]] in self.numeric_dp:

                                    value = self.numeric_dp[self.mappings["properties"][best_action_3[0]]]

                                    if (best_action_6[0] == 0) or (best_action_6[0] == 1):

                                        resteriction = 'min_inc'
                                    else:
                                        resteriction = 'max_inc'
                                    self.expression = create_numeric_data_some_value(
                                        self.mappings["properties"][best_action_3[0]],
                                        value, resteriction)
                            except:
                                pass
                else:
                    if not isinstance(self.mappings["properties"][best_action_3[0]], OWLDataPropertyExpression):
                        if best_action_6[0] == 1:
                            self.expression = quantifier(
                                self.expression,
                                self.mappings["properties"][best_action_3[0]],
                            )
                        if best_action_6[0] == 2:
                            self.expression = cardinality(
                                self.expression,
                                self.mappings["properties"][best_action_3[0]],
                                best_action_9[0]
                            )
                    else:
                        if self.mappings["properties"][
                            best_action_3[0]] in self.boolean_dp:  # TODO: create booldp dictionary
                            value = list(self.kb.get_data_property_values(self.pos_example,
                                                                          self.boolean_dp[self.mappings["properties"][
                                                                              best_action_3[0]].reminder]))[0]
                            if (str(value).lower() == 'false') or (str(value).lower() == '0'):
                                bool_exp = OWLDataHasValue(self.mappings["properties"][best_action_3[0]],
                                                           OWLLiteral(False))
                            else:
                                bool_exp = OWLDataHasValue(self.mappings["properties"][best_action_3[0]],
                                                           OWLLiteral(True))
                            self.expression = set_operators(
                                (
                                    self.expression,
                                    bool_exp))
                        if self.mappings["properties"][best_action_3[0]] in self.numeric_dp:

                            value = self.numeric_dp[self.mappings["properties"][
                                best_action_3[0]]]  # TODO: create numericdp dictionary
                            if value == None:
                                value = 0
                            if (best_action_6[0] == 0) or (best_action_6[0] == 1):
                                resteriction = 'min_inc'
                            else:
                                resteriction = 'max_inc'
                                numeric_exp = create_numeric_data_some_value(
                                    self.mappings["properties"][best_action_3[0]],
                                    value, resteriction)
                                if best_action_4[0] == 2:
                                    self.expression = set_operators(self.expression)
                                else:

                                    self.expression = set_operators((
                                        self.expression,
                                        numeric_exp))

            # reward = 0
            try:
                self.f1_score = evaluate_concept(self.kb, self.expression, F1(), self.lp.encode_kb(self.kb)).q
                info['f1'] = self.f1_score
                self.reward = self.f1_score
            except:
                self.expression = old_expression
                self.reward = -1
                # self.reward = self.f1_score + (1 / concept_len(self.expression))
            if (self.f1_score > 0.8) and (self.f1_score < 0.9):
                self.export_expr(
                    "80_90_Top_DL.txt", owl_expression_to_dl(self.expression) + '\n' + str(self.f1_score)
                )
            elif (self.f1_score > 0.9) and (self.f1_score < 0.95):
                self.export_expr(
                    "90_95_Top_DL.txt", owl_expression_to_dl(self.expression) + '\n' + str(self.f1_score)
                )
            elif self.f1_score > 95:
                self.export_expr("above_95_Top_DL.txt",
                                 owl_expression_to_dl(self.expression) + '\n' + str(self.f1_score))

            else:
                self.export_expr("below_80_Top_DL.txt",
                                 owl_expression_to_dl(self.expression) + '\n' + str(self.f1_score))

        self.rewards = {"a_0": self.reward, "a_1": self.reward}
        observations = {a: self._get_observation_state() for a in self.agents}

        infos = {'a_0': info, 'a_1': info}
        if any(terminations.values()) or all(truncations.values()):
            self.agents = []

        return observations, self.rewards, terminations, truncations, infos

    def constrained_action_sample(self, a1, a2):
        # should be okay since we are implementing a shared policy network for both agents for stability;
        # thus action logits will be obtained from the same distribution
        if len(set(a1).intersection(set(a2))) == 0:
            a1[0] = a2[0]
        return a1, a2

    def init_expression(self):
        candidate_exp = []
        direct_parent_conc = set()
        direct_parent_obj_prop = set()
        direct_parent_data_prop = set()

        for i in self.kb.concepts:
            conc_list = list(self.kb.reasoner.instances(i))
            self.pos_example = np.random.choice(list(self.pos))
            if self.pos_example in conc_list:
                direct_parent_conc.add(i)
                candidate_exp.append(i)
                if check_generators(self.kb.get_object_properties_for_ind(self.pos_example)) != False:
                    for x in self.kb.get_object_properties_for_ind(self.pos_example):
                        direct_parent_obj_prop.add(x)

                if check_generators(self.kb.get_data_properties_for_ind(self.pos_example)) != False:
                    for x in self.kb.get_data_properties_for_ind(self.pos_example):
                        direct_parent_data_prop.add(x)

                if check_generators(self.kb.get_boolean_data_properties()) != False:
                    for x in self.kb.get_boolean_data_properties():
                        if check_generators(self.kb.get_data_property_values(self.pos_example, x)) == False:
                            candidate_exp.append(OWLDataHasValue(x, OWLLiteral(False)))
                            candidate_exp.append(OWLDataHasValue(x, OWLLiteral(True)))
                        else:
                            value = list(self.kb.get_data_property_values(self.pos_example, x))[0]
                            candidate_exp.append(OWLDataHasValue(x, value))

        if len(direct_parent_conc) > 1:
            exp = self.kb.generator.union([x for x in direct_parent_conc])
            candidate_exp.append(exp)
            for i in direct_parent_obj_prop:
                candidate_exp.append(self.kb.generator.existential_restriction(exp, i))
                candidate_exp.append(self.kb.generator.universal_restriction(exp, i))
        else:
            if len(direct_parent_conc) >0:
                exp = self.kb.generator.existential_restriction([x for x in direct_parent_conc][0],
                                                            direct_parent_obj_prop.pop())
                n_props = np.random.randint(0, len(direct_parent_obj_prop) + 1)
            else:
                n_props = 0
            if n_props == 0:
                pass
            else:
                props = list(direct_parent_obj_prop)
                for i in range(n_props):
                    exp = self.kb.generator.existential_restriction(exp, props[i])
                    candidate_exp.append(exp)
                for i in range(n_props):
                    exp = self.kb.generator.universal_restriction(exp, props[i])
                    candidate_exp.append(exp)
        if n_props == 0:
            pass
        else:
            candidate_exp.append(exp)

        for i in self.kb.get_numeric_data_properties():
            try:
                value = self.numeric_dp[i]
                candidate_exp.append(create_numeric_data_some_value(i, value, 'min_inc'))
                candidate_exp.append(create_numeric_data_some_value(i, value, 'max_inc'))

            except:
                value = 1.0
                candidate_exp.append(create_numeric_data_some_value(i, value, 'min_inc'))
                candidate_exp.append(create_numeric_data_some_value(i, value, 'max_inc'))

        exp = candidate_exp[0]
        for i in candidate_exp:
            if evaluate_concept(self.kb, i, F1(), self.lp.encode_kb(self.kb)).q > evaluate_concept(self.kb, exp, F1(),
                                                                                                   self.lp.encode_kb(
                                                                                                           self.kb)).q:
                exp = i
        return np.random.choice([exp, self.cg.thing])

    def split_catboost(self, x, y):

        if np.issubdtype(x.dtype, np.number):

            x = np.array(x).reshape(-1, 1)
            y = np.array(y)
            try:
                pool = Pool(x, y)
                model = CatBoostClassifier(iterations=1, depth=1, logging_level='Silent')
                model.fit(pool)

                with tempfile.NamedTemporaryFile(delete=False, suffix='.json') as tmp:
                    tmp_path = tmp.name

                model.save_model(tmp_path, format='json')

                with open(tmp_path, 'r') as f:
                    model_json = json.load(f)
                global s
                s = model_json
                split_value = model_json['oblivious_trees'][0]['splits'][0]['border']
                os.remove(tmp_path)
                return split_value
            except:
                pass
                return
        else:
            return

    def _get_numeric_splits(self):
        dp_dict = dict(
            zip([x.iri.reminder for x in list(self.kb.get_data_properties())], list(self.kb.get_data_properties())))
        x, y = self._get_data_prop()

        dp_numeric = {}
        for i in x.columns:
            try:
                dp_numeric[dp_dict[x[i].name]] = self.split_catboost(x[i], y)
            except:
                pass
        return dp_numeric

    def render(self):
        print(self.expression.str)
        return

    def export_expr(self, file_name, value):
        with open(f"{file_name}.txt", "a", encoding="utf-8") as myfile:
            myfile.write(value + "\n\n")

    def _get_data_prop(self):
        pos_vals = defaultdict(list)
        neg_vals = defaultdict(list)

        for j in self.kb.get_data_properties():
            for i in self.pos:
                prop = list(self.kb.get_data_property_values(i, j))

                if len(prop) > 0:
                    for k in range(len(prop)):
                        try:
                            pos_vals[j.iri.reminder].append(float(prop[k].get_literal()))
                        except:
                            pos_vals[j.iri.reminder].append(prop[k].get_literal())

            for i in self.neg:
                prop = list(self.kb.get_data_property_values(i, j))
                if len(prop) > 0:
                    for k in range(len(prop)):
                        try:
                            neg_vals[j.iri.reminder].append(float(prop[k].get_literal()))
                        except:
                            neg_vals[j.iri.reminder].append(prop[k].get_literal())

        pos_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in pos_vals.items()]))

        pos_df['target'] = 1

        neg_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in neg_vals.items()]))

        neg_df['target'] = 0

        df = pd.concat([pos_df, neg_df], axis=0)
        df = df.fillna('nan')

        x = df.iloc[:, :-1]
        y = df['target']

        return x, y

    def _get_split_catboost(self):
        x, y = self._get_data_prop()
        data_prop_splits = {}
        for i in x.select_dtypes(include='number').columns:
            data_prop_splits[i] = self.split_catboost(x[i], y)

        return data_prop_splits

    @functools.lru_cache(maxsize=None)
    def observation_space(self, agent):
        return spaces.Box(low=-1, high=1, shape=(1, 128)
                          )

    @functools.lru_cache(maxsize=None)
    def action_space(self, agent):
        return spaces.Dict(
            {
                "actions": spaces.MultiDiscrete(
                    [
                        len(self.concepts),
                        len(self.concepts),

                        len(self.concepts),
                        len(self.concepts),

                        len(self.properties),
                        len(self.properties),

                        len(list(self.set_operators)),
                        len(list(self.set_operators)),

                        len(list(self.quantifiers)),
                        len(list(self.quantifiers)),

                        len(list(self.cardinality)),
                        len(list(self.cardinality)),

                        3,  # flag
                        3,  # flag

                        2,  # stopping
                        2,  # stopping

                        10,
                        10
                    ]
                ),
                "masses": spaces.Box(0, 1, (9, 15))})

# Helper classes

In [6]:
import heapq
import itertools
from collections import  defaultdict
class Node:
    def __init__(self, individual, hop, parent=None, neighbors=None, k=None):
        self.individual = individual
        self.hop = hop
        self.parent = parent
        self.children = []
        self.visits = 0
        self.total_reward = 0
        if hop < k and neighbors:
            self.untried_neighbors = set(n for _, n in neighbors.get(individual, []))
        else:
            self.untried_neighbors = set()


class Hypothesis_Cache:
    def __init__(self, n):
        self.n = n
        self.hypo_dict = defaultdict(list)

    def _add_hypo(self, expression, score):
        best_score = 0
        if score > best_score:
            self.hypo_dict['exp'].append(expression)
            self.hypo_dict['score'].append(score)

        if len(self.hypo_dict) > 0:
            combined = list(zip(self.hypo_dict['exp'],
                                self.hypo_dict['score']))
            sorted_hypothesis = sorted(combined, key=lambda x: x[1], reverse=True)
            if len(sorted_hypothesis) > self.n:
                del sorted_hypothesis[-1]
            sorted_exp, sorted_score = zip(*sorted_hypothesis)

            self.hypo_dict['exp'] = list(sorted_exp)
            self.hypo_dict['score'] = list(sorted_score)

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical
from torch.distributions.normal import Normal
from collections import defaultdict
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import numpy as np
import time



class Lyra(nn.Module):
    name = 'Lyra'

    def __init__(self, knoweldge_base, pos, neg, run_name='R1', n_hypothesis = 5):
        super().__init__()
        self.multi_a_env = Concept_Dec_POMPD(knoweldge_base, pos, neg, "f1",
                                        10)

        self.out_dim_cat = self.multi_a_env.action_space('a_0')['actions'].nvec.sum()
        self.out_dim_cont = np.prod(self.multi_a_env.action_space('a_0')['masses'].shape)
        self.network = nn.Sequential(
            layer_init(nn.Linear(64, 64)),
            nn.ReLU(),
            layer_init(nn.Linear(64, 64)),
            nn.ReLU(),
            layer_init(nn.Linear(64, 64)),
            nn.ReLU(),
        )
        self.run_name = run_name
        self.cat_actor = layer_init(nn.Linear(64, self.out_dim_cat), std=0.01)
        self.cont_actor = layer_init(nn.Linear(self.out_dim_cat,
                                               self.out_dim_cont), std=0.01)

        self.critic = layer_init(nn.Linear(64, 1), std=1)

        torch.backends.cudnn.deterministic = False
        # self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.device = 'cpu'
        self.log_std = nn.Parameter(torch.zeros(self.out_dim_cont))
        # self.writer = SummaryWriter(f"runs/{self.run_name}")
        self.session_info = defaultdict(list)
        self.hypothesis = Hypothesis_Cache(n=n_hypothesis)

    def get_value(self, x):
        x = x.clone()
        return self.critic(self.network(x))

    def get_action_and_value(self, x, action=None):

        hidden = self.network(x)
        cat_action_logits = self.cat_actor(hidden)
        cont_action_logits = self.cont_actor(cat_action_logits)

        cat_actions_dim = self.multi_a_env.action_space("a_0")["actions"].nvec
        masses_action_dim = self.multi_a_env.action_space("a_0")["masses"].shape

        is_batch = cat_action_logits.dim() == 2
        if not is_batch:
            action_std = self.log_std.exp()

            bpa_props = Normal(
                cont_action_logits.reshape(
                    masses_action_dim
                ),
                action_std.reshape(masses_action_dim),
            )

        else:
            action_std = self.log_std.exp()
            bpa_props = Normal(loc=cont_action_logits.view(
                (cont_action_logits.size(0), masses_action_dim[0] * masses_action_dim[1])), scale=action_std)

        splits = torch.split(
            cat_action_logits,
            cat_actions_dim.tolist(),
            dim=(0 if not is_batch else 1),
        )
        cat_probs = [Categorical(logits=s) for s in splits]

        if action is None:
            cat_actions_a1 = torch.stack(
                [dist.sample() for dist in cat_probs], dim=1 if is_batch else 0
            )
            cat_actions_a2 = torch.stack(
                [dist.sample() for dist in cat_probs], dim=1 if is_batch else 0
            )
            masses_action_a1 = bpa_props.sample()
            masses_action_a2 = bpa_props.sample()
            log_propbs_cat_a_1 = torch.stack(
                [
                    cat_probs[x].log_prob(cat_actions_a1[..., x])
                    for x in range(len(cat_probs))
                ],
                dim=-1,
            ).sum(-1)
            log_propbs_cat_a_2 = torch.stack(
                [
                    cat_probs[x].log_prob(cat_actions_a2[..., x])
                    for x in range(len(cat_probs))
                ],
                dim=-1,
            ).sum(-1)
            if not is_batch:
                action = {
                    "a_0": {
                        "actions": cat_actions_a1,
                        "masses": masses_action_a1.reshape(*masses_action_dim),
                    },
                    "a_1": {
                        "actions": cat_actions_a2,
                        "masses": masses_action_a2.reshape(*masses_action_dim),
                    },
                }
        else:
            (
                cat_actions_a1,
                cat_actions_a2,
                masses_action_a1,
                masses_action_a2,
            ) = torch.split(
                action, [18, 18, 135, 135], dim=-1
            )  # TODO: Better splitting

            log_propbs_cat_a_1 = torch.stack(
                [
                    cat_probs[x].log_prob(cat_actions_a1[..., x])
                    for x in range(len(cat_probs))
                ],
                dim=-1,
            ).sum(-1)
            log_propbs_cat_a_2 = torch.stack(
                [
                    cat_probs[x].log_prob(cat_actions_a2[..., x])
                    for x in range(len(cat_probs))
                ],
                dim=-1,
            ).sum(-1)
        bpa_log_props_a1 = (
            bpa_props.log_prob(masses_action_a1)
            .sum(-1)
        )
        bpa_log_probs_a2 = (
            bpa_props.log_prob(masses_action_a2)
            .sum(-1)
        )

        if is_batch:
            aggregated_log_probs = (log_propbs_cat_a_1.sum(-1) + log_propbs_cat_a_2.sum(-1) + bpa_log_props_a1.sum(
                -1) + bpa_log_probs_a2.sum(-1))
        else:
            aggregated_log_probs = (
                    log_propbs_cat_a_1
                    + log_propbs_cat_a_2
                    + bpa_log_props_a1.sum(-1)
                    + bpa_log_probs_a2.sum(-1)
            )

        total_entropy = self.multi_a_env.nugyen_entropy

        if not is_batch:
            action_for_buffer = torch.cat(
                [
                    cat_actions_a1,
                    cat_actions_a2,
                    masses_action_a1.flatten(
                        start_dim=-(len(masses_action_dim))
                    ).view(-1),
                    masses_action_a2.flatten(
                        start_dim=-(len(masses_action_dim))
                    ).view(-1),
                ],
                dim=-1,
            )

        else:
            action_for_buffer = torch.cat(
                [
                    cat_actions_a1.squeeze(1),
                    cat_actions_a2.squeeze(1),
                    masses_action_a1.reshape(
                        -1, (masses_action_dim[0] * masses_action_dim[1])
                    ),
                    masses_action_a2.reshape(
                        -1, (masses_action_dim[0] * masses_action_dim[1])
                    ),
                ],
                dim=-1,
            )

        return (
            action,
            aggregated_log_probs,
            total_entropy,
            self.critic(hidden),
            action_for_buffer,
        )

    def best_hypotheses(self):
        return self.hypothesis.hypo_dict['exp']

    def fit(self, gamma=0.99, gae_lambda=0.95, batch_size=10, minibatch_size=32, ent_coef=0.01, vf_coef=0.5,
            target_kl=None,
            update_epochs=4, n_steps=256, learning_rate=0.001, anneal_lr=True, n_hypothesis=20):

        print('Start learning...')

        optimizer = optim.Adam(self.parameters(), lr=learning_rate, eps=1e-5)
        global_step = 0
        self.start_time = time.time()

        next_obs, info = self.multi_a_env.reset()
        next_obs = next_obs.to(self.device)

        next_termination = torch.zeros(1).to(self.device)
        next_truncation = torch.zeros(1).to(self.device)
        num_updates = n_steps // batch_size
        obs = torch.zeros((n_steps, 64)).to(self.device)
        actions = torch.zeros((n_steps, 306)).to(self.device)  # TODO: subistitute the hardcoded)

        logprobs = torch.zeros((n_steps, 1)).to(self.device)
        rewards = torch.zeros((n_steps, 1)).to(self.device)
        terminations = torch.zeros((n_steps, 1)).to(self.device)
        truncations = torch.zeros((n_steps, 1)).to(self.device)
        values = torch.zeros((n_steps, 1)).to(self.device)

        for update in range(1, num_updates + 1):
            if anneal_lr:
                frac = 1.0 - (update - 1.0) / num_updates
                lrnow = frac * learning_rate
                optimizer.param_groups[0]["lr"] = lrnow
            for step in range(0, n_steps):
                global_step += 1
                obs[step] = next_obs
                terminations[step] = next_termination
                truncations[step] = next_truncation
                # ALGO LOGIC: action logic
                with torch.no_grad():
                    action, logprob, _, value, action_for_buffer = (
                        self.get_action_and_value(next_obs.detach())
                    )
                    values[step] = value.flatten()
                actions[step] = action_for_buffer
                logprobs[step] = logprob
                action['a_0']['actions'] = action['a_0']['actions'].numpy()
                action['a_0']['masses'] = action['a_0']['masses'].numpy()
                action['a_1']['actions'] = action['a_1']['actions'].numpy()
                action['a_1']['masses'] = action['a_1']['masses'].numpy()

                next_obs, reward, termination, truncation, info = self.multi_a_env.step(action)
                self.hypothesis._add_hypo(self.multi_a_env.expression, self.multi_a_env.f1_score)
                self.conflict = self.multi_a_env.conflict
                self.pl_bel_gap = self.multi_a_env.pl_bel_gaps

                self.session_info['conflict'].append(self.conflict)
                self.session_info['pl_bl_gap'].append(self.pl_bel_gap)
                self.session_info['reward'].append(reward)
                

                rewards[step] = (torch.tensor(reward["a_0"]).to(self.device).view(-1))

                next_obs = self.multi_a_env._get_observation_state().to(self.device)
                next_termination = torch.Tensor([1 if termination["a_0"] else 0]).to(self.device)
                next_truncation = torch.Tensor([1 if termination["a_0"] else 0]).to(self.device)

            # bootstrap value if not done
            with torch.no_grad():
                next_value = self.get_value(next_obs.detach()).reshape(1, -1)
                advantages = torch.zeros_like(rewards).to(self.device)
                lastgaelam = 0
                next_done = torch.maximum(next_termination, next_truncation)
                dones = torch.maximum(terminations, truncations)
                for t in reversed(range(n_steps)):
                    if t == n_steps - 1:
                        nextnonterminal = 1.0 - next_done
                        nextvalues = next_value
                    else:
                        nextnonterminal = 1.0 - dones[t + 1]
                        nextvalues = values[t + 1]
                    delta = (
                            rewards[t] + gamma * nextvalues * nextnonterminal - values[t]
                    )
                    advantages[t] = lastgaelam = (
                            delta + gamma * gae_lambda * nextnonterminal * lastgaelam
                    )
                returns = advantages + values

            # flatten the batch
            b_obs = obs.reshape((-1,) + (1, 64))
            b_logprobs = logprobs.reshape(-1)
            b_actions = actions.reshape((-1,) + (1, 306))
            b_advantages = advantages.reshape(-1)
            b_returns = returns.reshape(-1)
            b_values = values.reshape(-1)
            # Optimizing the policy and value network
            b_inds = np.arange(batch_size)
            for epoch in range(update_epochs):
                np.random.shuffle(b_inds)
                for start in range(0, batch_size, minibatch_size):
                    optimizer.zero_grad()
                    end = start + minibatch_size
                    mb_inds = b_inds[start:end]
                    _, newlogprob, entropy, newvalue, action_for_buffer = (
                        self.get_action_and_value(
                            b_obs[mb_inds].squeeze(1).detach(), b_actions[mb_inds].detach()
                        )
                    )
                    logratio = newlogprob - b_logprobs[mb_inds]
                    ratio = logratio.exp()
                    with torch.no_grad():
                        # calculate approx_kl http://joschu.net/blog/kl-approx.html
                        old_approx_kl = (-logratio).mean()
                        approx_kl = ((ratio - 1) - logratio).mean()
                    mb_advantages = b_advantages[mb_inds]
                    norm_adv = True
                    if norm_adv:
                        mb_advantages = (mb_advantages - mb_advantages.mean()) / (
                                mb_advantages.std() + 1e-8
                        )
                    pg_loss1 = -mb_advantages * ratio
                    pg_loss = pg_loss1.mean()
                    # Value loss
                    newvalue = newvalue.view(-1)
                    v_loss = 0.5 * ((newvalue - b_returns[mb_inds]) ** 2).mean()
                    if isinstance(entropy, int):
                        entropy_loss = entropy
                    else:
                        entropy_loss = entropy.mean()
                    loss = pg_loss - ent_coef * entropy_loss + v_loss * vf_coef
                    loss.backward()
                    optimizer.step()

                if target_kl is not None:
                    if approx_kl > target_kl:
                        break

        report_df = pd.DataFrame(self.session_info).T
        report_df.to_csv("session_out.csv", index=False)
        print("SPS:", int(global_step / (time.time() - self.start_time)))

In [ ]:
# from utils.utils import *
# from Lyra.lyra_agent import Lyra

from ontolearn.learners import OCEL, Drill, CELOE
from ontolearn.concept_learner import EvoLearner
import random
import numpy as np
import torch

Drill.name = 'Drill'

if __name__ == '__main__':
    rnd_seed = random.sample(range(1, 1000), 30)

    KGs = [r".../sml/carc/carcinogenesis.owl"]
    lps = [r".../sml/carc/"]
    print('==============================================================================')
    for idx, i in enumerate(KGs):
        random.seed(rnd_seed[idx])
        np.random.seed(rnd_seed[idx])
        torch.manual_seed(rnd_seed[idx])
        print('random_seed: ', rnd_seed[idx])
        print('==============================================================================')
        pos, neg = pos_and_neg_parser(lps[idx])
        experiment_report('carcinogenesis', [Lyra, EvoLearner, CELOE, Drill], 2,
                          i, pos, neg)

random_seed:  655
Start learning...
SPS: 2
trial:  0 model name:  Lyra f1:  0.71831 time:  3093.005 concept_len:  4 expression:  ≤ 4 hasAtom.Oxygen
f1:  0.71831 /n exp: ≤ 4 hasAtom.Oxygen
f1:  0.70306 /n exp: ⊤
f1:  0.70306 /n exp: ⊤
f1:  0.70796 /n exp: ∃ hasStructure.(¬Compound)
f1:  0.70796 /n exp: ∃ hasStructure.(¬Compound)
Start learning...
SPS: 2
trial:  1 model name:  Lyra f1:  0.71082 time:  3046.732 concept_len:  13 expression:  ≥ 4 hasBond.((¬(Compound ⊔ (∃ mouse_lymph.xsd:double[≤ 0]))) ⊔ Manganese-85 ⊔ Ar_halide)
f1:  0.71082 /n exp: ≥ 4 hasBond.((¬(Compound ⊔ (∃ mouse_lymph.xsd:double[≤ 0]))) ⊔ Manganese-85 ⊔ Ar_halide)
f1:  0.70306 /n exp: Compound
f1:  0.70306 /n exp: Compound
f1:  0.70925 /n exp: ≥ 3 hasBond.(¬Compound)
f1:  0.70925 /n exp: (≥ 3 hasBond.(¬Compound)) ⊔ (∃ mouse_lymph.xsd:double[≤ 0])
trial:  0 model name:  evolearner f1:  0.61922 time:  8.269 concept_len:  15 expression:  ((∃ salmonella.{true}) ⊔ (∃ isMutagenic.{false})) ⊓ ((∃ salmonella_n.{true}) ⊔ (∃ a